In [1]:
import os,sys
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import functions as fun

Load AR5 data

In [2]:
gdyn = {}
time = np.arange(2007,2101)
time_lt = np.arange(1901,2101)
with xr.open_dataset('../rawdata/component-ts-slr-45.nc') as ds:
    lon = ds['lon'].values
    lat = ds['lat'].values
    gia = ds['gia_m'].values
    grw = ds['grw_m'].values
    gdyn['rcp45'] = ds['greendyn_m'].values
with xr.open_dataset('../rawdata/component-ts-slr-85.nc') as ds:
    gdyn['rcp85'] = ds['greendyn_m'].values

GIA

In [3]:
#Annual change in cm
dgia = 100.*(gia[1,:,:]-gia[0,:,:])

#Create longterm field relative to 1996
gia_lt = np.zeros((len(time_lt),gia.shape[1],gia.shape[2]))
for t,tt in enumerate(time_lt):
    gia_lt[t,:,:]= dgia*(tt-1996)

#Save
slr2 = xr.DataArray(gia_lt,dims=('time_lt','lat','lon'),coords={'time_lt':time_lt,'lat':lat,'lon':lon})
ds = xr.Dataset({'SLR_lt':slr2})
ds.to_netcdf(f'../data/others/gia.nc',encoding={'SLR_lt':{'zlib': True, 'complevel': 9}})
ds.close()

Greenland dynamics

In [4]:
for scen in ['rcp45','rcp85']:
    #Projections in cm
    gd = 100.*gdyn[scen]

    #Boundary conditions for connect in 2007
    ddt = gd[1,:,:]-gd[0,:,:] #Rate
    dg = gd[0,:,:] #Value
    
    #third-order connect 1996-2007
    startyear = 1996
    dt = 2007-startyear
    tt = np.arange(dt)
    c = 3*dg/(dt**2)-ddt/dt
    d = ddt/(dt**2) - 2*dg/(dt**3)
    
    #Fill three time periods: zero, third-order, projection
    gdyn_lt = np.zeros((len(time_lt),gia.shape[1],gia.shape[2]))
    gdyn_lt[startyear-1901:startyear-1901+dt,:,:] = c[np.newaxis,:,:]*tt[:,np.newaxis,np.newaxis]**2 + d[np.newaxis,:,:]*tt[:,np.newaxis,np.newaxis]**3
    gdyn_lt[startyear-1901+dt:,:,:] = gd
    
    #Save
    slr2 = xr.DataArray(gdyn_lt,dims=('time_lt','lat','lon'),coords={'time_lt':time_lt,'lat':lat,'lon':lon})
    ds = xr.Dataset({'SLR_lt':slr2})
    ds.to_netcdf(f'../data/others/gdyn_{scen}.nc',encoding={'SLR_lt':{'zlib': True, 'complevel': 9}})
    ds.close()

Groundwater

In [5]:
#Projections in cm
gr = 100.*grw

#Increase 2007-2008
ddt = gr[1,:,:]-gr[0,:,:]
dg = gr[0,:,:]

#Acceleration 1996-2007
startyear = 1996
dt = 2007-startyear
tt = np.arange(dt)

c = 3*dg/(dt**2)-ddt/dt
d = ddt/(dt**2) - 2*dg/(dt**3)

#Fill three time periods: zero, accel, projection
grw_lt = np.zeros((len(time_lt),gia.shape[1],gia.shape[2]))
grw_lt[startyear-1901:startyear-1901+dt,:,:] = c[np.newaxis,:,:]*tt[:,np.newaxis,np.newaxis]**2 + d[np.newaxis,:,:]*tt[:,np.newaxis,np.newaxis]**3
grw_lt[startyear-1901+dt:,:,:] = gr

#Save
slr2 = xr.DataArray(grw_lt,dims=('time_lt','lat','lon'),coords={'time_lt':time_lt,'lat':lat,'lon':lon})
ds = xr.Dataset({'SLR_lt':slr2})
ds.to_netcdf(f'../data/others/grw.nc',encoding={'SLR_lt':{'zlib': True, 'complevel': 9}})
ds.close()